In [50]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import time
import numpy as np

class Parser:
    """Parameters for training"""
    def __init__(self):
        self.epochs = 10
        self.lr = 0.00001
        self.test_batch_size = 8
        self.batch_size = 8
        self.log_interval = 10
        self.seed = 1
    
args = Parser()

torch.manual_seed(args.seed)
kwargs = {}

In [51]:
with open('./data/boston_housing.pickle','rb') as f:
    ((X, y), (X_test, y_test)) = pickle.load(f)

X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()
# preprocessing
mean = X.mean(0, keepdim=True)
dev = X.std(0, keepdim=True)
mean[:, 3] = 0.
dev[:, 3] = 1.
X = (X - mean) / dev
X_test = (X_test - mean) / dev
train = TensorDataset(X, y)
test = TensorDataset(X_test, y_test)
train_loader = DataLoader(train, batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = DataLoader(test, batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [52]:
X.shape

torch.Size([404, 13])

In [53]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 32)
        self.fc2 = nn.Linear(32, 24)
        self.fc4 = nn.Linear(24, 16)
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = x.view(-1, 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc4(x))
        x = self.fc3(x)
        return x

In [54]:
import syft as sy

hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
james = sy.VirtualWorker(hook, id="james")

compute_nodes = [bob, alice]

In [55]:
train_distributed_dataset = []

for batch_idx, (data,target) in enumerate(train_loader):
    data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
    target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
    train_distributed_dataset.append((data, target))

In [56]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=args.lr)

In [57]:
def train(epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(train_distributed_dataset):
        worker = data.location
        model.send(worker)

        optimizer.zero_grad()
        # update the model
        pred = model(data)
        loss = F.mse_loss(pred.view(-1), target)
        loss.backward()
        optimizer.step()
        model.get()
            
        if batch_idx % args.log_interval == 0:
            loss = loss.get()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * data.shape[0], len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))

In [58]:
def test():
    model.eval()
    test_loss = 0
    for data, target in test_loader:
        output = model(data)
        test_loss += F.mse_loss(output.view(-1), target, reduction='sum').item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}\n'.format(test_loss))

In [59]:
t = time.time()

for epoch in range(1, args.epochs + 1):
    train(epoch)

    
total_time = time.time() - t
print('Total', round(total_time, 2), 's')

Train Epoch: 1 [0/51 (0%)]	Loss: 694.438599
Train Epoch: 1 [80/51 (20%)]	Loss: 720.955017
Train Epoch: 1 [160/51 (39%)]	Loss: 728.808655
Train Epoch: 1 [240/51 (59%)]	Loss: 887.254028
Train Epoch: 1 [320/51 (78%)]	Loss: 457.007721
Train Epoch: 1 [200/51 (98%)]	Loss: 329.091431
Train Epoch: 2 [0/51 (0%)]	Loss: 692.727478
Train Epoch: 2 [80/51 (20%)]	Loss: 719.209351
Train Epoch: 2 [160/51 (39%)]	Loss: 727.093750
Train Epoch: 2 [240/51 (59%)]	Loss: 885.298889
Train Epoch: 2 [320/51 (78%)]	Loss: 455.514587
Train Epoch: 2 [200/51 (98%)]	Loss: 327.846405
Train Epoch: 3 [0/51 (0%)]	Loss: 691.022095
Train Epoch: 3 [80/51 (20%)]	Loss: 717.469299
Train Epoch: 3 [160/51 (39%)]	Loss: 725.386047
Train Epoch: 3 [240/51 (59%)]	Loss: 883.346619
Train Epoch: 3 [320/51 (78%)]	Loss: 454.032349
Train Epoch: 3 [200/51 (98%)]	Loss: 326.606903
Train Epoch: 4 [0/51 (0%)]	Loss: 689.318970
Train Epoch: 4 [80/51 (20%)]	Loss: 715.744141
Train Epoch: 4 [160/51 (39%)]	Loss: 723.682068
Train Epoch: 4 [240/51 (59%)]

In [60]:
test()


Test set: Average loss: 602.1180



In [61]:
remote_dataset = (list(), list())

train_distributed_dataset = []

for batch_idx, (data,target) in enumerate(train_loader):
    data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
    target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
    remote_dataset[batch_idx % len(compute_nodes)].append((data, target))

def update(data, target, model, optimizer):
    model.send(data.location)
    optimizer.zero_grad()
    pred = model(data)
    loss = F.mse_loss(pred.view(-1), target)
    loss.backward()
    optimizer.step()
    return model

bobs_model = Net()
alices_model = Net()

bobs_optimizer = optim.SGD(bobs_model.parameters(), lr=args.lr)
alices_optimizer = optim.SGD(alices_model.parameters(), lr=args.lr)

models = [bobs_model, alices_model]
params = [list(bobs_model.parameters()), list(alices_model.parameters())]
optimizers = [bobs_optimizer, alices_optimizer]

In [62]:
import copy

def train():
    for data_index in range(len(remote_dataset[0])-1):
        
        # update remote models
        for remote_index in range(len(compute_nodes)):
            data, target = remote_dataset[remote_index][data_index]
            models[remote_index] = update(data, target, models[remote_index], optimizers[remote_index])

        # encrypted aggregation
        new_params = list()
        for param_i in range(len(params[0])):
            spdz_params = list()
            for remote_index in range(len(compute_nodes)):
                copy_of_parameter = copy.copy(params[remote_index][param_i])
#                 fixed_precision_param = copy_of_parameter.fix_prec()
                encrypted_param = copy_of_parameter.share(bob, alice, crypto_provider=james)
                param = encrypted_param.get()
                spdz_params.append(param)
#                 spdz_params.append(params[remote_index][param_i].copy().fix_precision().share(bob, alice, crypto_provider=james).get())
            new_param = (spdz_params[0] + spdz_params[1]).get().float()/2
            new_params.append(new_param)

        # cleanup
        with torch.no_grad():
            for model in params:
                for param in model:
                    param *= 0

            for model in models:
                model.get()

            for remote_index in range(len(compute_nodes)):
                for param_index in range(len(params[remote_index])):
                    params[remote_index][param_index].set_(new_params[param_index])

In [63]:
def test():
    models[1].eval()
    test_loss = 0
    for data, target in test_loader:
        output = models[0](data)
        test_loss += F.mse_loss(output.view(-1), target, reduction='sum').item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}\n'.format(test_loss))

In [64]:
t = time.time()

for epoch in range(args.epochs):
    print(f"Epoch {epoch + 1}")
    train()
    test()

    
total_time = time.time() - t
print('Total', round(total_time, 2), 's')

Epoch 1
Test set: Average loss: 615.8578

Epoch 2
Test set: Average loss: 615.8579

Epoch 3
Test set: Average loss: 615.8578

Epoch 4
Test set: Average loss: 615.8578

Epoch 5
Test set: Average loss: 615.8578

Epoch 6
Test set: Average loss: 615.8578

Epoch 7
Test set: Average loss: 615.8578

Epoch 8
Test set: Average loss: 615.8579

Epoch 9
Test set: Average loss: 615.8578

Epoch 10
Test set: Average loss: 615.8578

Total 18.44 s
